## View-Factor debug

This file is thought of a debug file for the viewfactor, it creates a small example of five points p(x,y) and creates the view matrix. It prints the matrices of all temporal variables during the calculation of the viewFactor

_calc_vector(value) creates a vector between 2 points
normal(x,y) creates a normal vector to a surface defined by 2 points
calc_viewfactor(x,y) creates the view-factor for a surface

In [1]:
import numpy as np

def normal(x, y):
    """Calc normal vector and return it."""
    x = np.concatenate(([x[0]], x, [x[-1]]))
    y = np.concatenate(([y[0]], y, [y[-1]]))
    dx = _calc_vector(x)
    dy = _calc_vector(y)
    length = np.linalg.norm([dx, dy], axis=0)
    return dy / length, -dx / length

def _calc_vector(value):
    """Subtract end coordinate with start coordinate.

    :param value: coordinate array
    """
    delta = value[2:] - value[:-2]
    return delta

def calc_viewfactor(x, y):
    """Calculates the view-factor from surface parameters

    :return: nxn matrix representing the view-factor
    """
    # create arrays for nodes i & j (j has equal rows, i has equal columns)
    x_j = np.ones(shape=(x.size, x.size)) * x
    y_j = np.ones(shape=(y.size, y.size)) * y

    x_i = x_j.transpose()
    y_i = y_j.transpose()

    print("x_i: \n{}\n".format(x_i))
    print("y_i: \n{}\n".format(y_i))
    print("x_j: \n{}\n".format(x_j))
    print("y_j: \n{}\n".format(y_j))
    
    # calculate distances between nodes i & j
    x_ij = x_i - x_j
    y_ij = y_i - y_j

    # create arrays for the normal vectors of nodes i & j (j is transposed)
    x_normal, y_normal = normal(x, y)
    x_i_normal = np.ones_like(x_i) * -x_normal[:, np.newaxis]
    y_i_normal = np.ones_like(y_i) * -y_normal[:, np.newaxis]
    x_j_normal = np.ones_like(x_j) * -x_normal
    y_j_normal = np.ones_like(y_j) * -y_normal
    
    print("x_i_normal: \n{}\n".format(x_i_normal))
    print("y_i_normal: \n{}\n".format(y_i_normal))
    print("x_j_normal: \n{}\n".format(x_j_normal))
    print("y_j_normal: \n{}\n".format(y_j_normal))

    # calculate cosines of angles (cos_a, cos_b) with [(a*b)/(|a|*|b|)]
    # deactivate warnings when division by zero -> is covered with np.nan_to_num
    with np.errstate(divide='ignore', invalid='ignore'):
        cos_a = np.nan_to_num((x_j_normal * x_ij + y_j_normal * y_ij) / (np.sqrt(x_j_normal ** 2 + y_j_normal ** 2) * np.sqrt(x_ij ** 2 + y_ij ** 2)))
        cos_b = np.nan_to_num((x_i_normal * -x_ij + y_i_normal * -y_ij) / (np.sqrt(x_i_normal ** 2 + y_i_normal ** 2) * np.sqrt(x_ij ** 2 + y_ij ** 2)))

    print("cos_a: \n{}\n".format(cos_a))
    print("cos_b: \n{}\n".format(cos_b))

    # calculate distances between nodes i & j (d_ij)
    d_ij = np.sqrt(x_ij ** 2 + y_ij ** 2)

    # calculate surface length of node (delta_l)
    x_ext = np.concatenate(([2 * x[0] - x[1]], x, [3 * x[-1] - x[-2]]))
    y_ext = np.concatenate(([y[0]], y, [y[-1]]))
    d_x = x_ext[1:] - x_ext[:-1]
    d_y = y_ext[1:] - y_ext[:-1]
    d_l = np.sqrt(d_x ** 2 + d_y ** 2)
    d_l_avg = (d_l[1:] + d_l[:-1]) / 2
    delta_l = np.ones_like(x_i) * d_l_avg

    # calculate view-factor and mask out all values where cos_a<0 and cos_b<0 and all elements on diagonal
    # deactivate warnings when division by zero ->is covered with np.nan_to_num
    with np.errstate(divide='ignore', invalid='ignore'):
        f_ij = np.nan_to_num((cos_a * cos_b * delta_l) / (2 * d_ij))

    print("\nf_ij: \n{}\n".format(f_ij))
    mask = (cos_a > np.zeros_like(x_i)) * (cos_b > np.zeros_like(x_i)) * np.invert(np.eye(x.size, dtype=bool))
    print("(cos_a > np.zeros_like(x_i)): \n{}\n".format(cos_a > np.zeros_like(x_i)))
    print("(cos_b > np.zeros_like(x_i)): \n{}\n".format(cos_b > np.zeros_like(x_i)))
    print("np.invert(np.eye(x.size, dtype=bool)): \n{}\n".format(np.invert(np.eye(x.size, dtype=bool))))
    print("mask: \n{}\n".format(mask))

    return f_ij*mask

define 2 matrices x, y which represent 5 points of a surface

In [2]:
x = np.array([-2.0,-1.0,0.0,1.0,2.0])
y = np.array([0.0,-2.0,-3.0,-2.0,0.0])

In [3]:
print("x: \n{}\n".format(x))
print("y: \n{}\n".format(y))

x: 
[-2. -1.  0.  1.  2.]

y: 
[ 0. -2. -3. -2.  0.]



calculate the viewfactor and print out all temporal variables in the function, whereas:
-  x_i, y_i are the points where redeposition is happening (point of interest)
-  x_j, y_j are all points from where the redeposition is coming from
-  x_i_normal, y_i_normal, x_j_normal, y_j_normal are the normal vectors of the surfaces in the points i and j
-  cos_a is the angle between the normal vector in i and the connecting line between i and j
-  cos_b is the angle between the normal vector in j and the connecting line between i and j
-  f_ij is the view_factor without considering the invisible surfaces
- mask is the mask for the viewfactor to unmask all invisible surfaces (false is for invisible, true is for visible)
-  viewfactor is the returned correct viewfactor

In [4]:
viewfactor = calc_viewfactor(x,y)
print("viewfactor: \n{}".format(viewfactor))

x_i: 
[[-2. -2. -2. -2. -2.]
 [-1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.]]

y_i: 
[[ 0.  0.  0.  0.  0.]
 [-2. -2. -2. -2. -2.]
 [-3. -3. -3. -3. -3.]
 [-2. -2. -2. -2. -2.]
 [ 0.  0.  0.  0.  0.]]

x_j: 
[[-2. -1.  0.  1.  2.]
 [-2. -1.  0.  1.  2.]
 [-2. -1.  0.  1.  2.]
 [-2. -1.  0.  1.  2.]
 [-2. -1.  0.  1.  2.]]

y_j: 
[[ 0. -2. -3. -2.  0.]
 [ 0. -2. -3. -2.  0.]
 [ 0. -2. -3. -2.  0.]
 [ 0. -2. -3. -2.  0.]
 [ 0. -2. -3. -2.  0.]]

x_i_normal: 
[[ 0.89442719  0.89442719  0.89442719  0.89442719  0.89442719]
 [ 0.83205029  0.83205029  0.83205029  0.83205029  0.83205029]
 [-0.         -0.         -0.         -0.         -0.        ]
 [-0.83205029 -0.83205029 -0.83205029 -0.83205029 -0.83205029]
 [-0.89442719 -0.89442719 -0.89442719 -0.89442719 -0.89442719]]

y_i_normal: 
[[0.4472136 0.4472136 0.4472136 0.4472136 0.4472136]
 [0.5547002 0.5547002 0.5547002 0.5547002 0.5547002]
 [1.        1.        1.        1.        1.       ]
 [0.55